<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance用オーダー発行API

`BinanceMarket`は、通過ペアー毎に以下の機能を提供します。

* 蓄積用データベースの生成（Sqlite)
* 約定データのダウンロードと更新
  * 過去データのダウンロード(Binance Webサイト)
  * リアルタイム更新(WebSocket)
* 約定データのDBからの取得と足の作成
  * 任意の期間の足の作成(`ohlcv`メソッド)
  * VAP(Value At Price)の計算(`vap`メソッド)
* オーダーの発行・キャンセル（REST API)　KEYとSECRETの設定が必要です。

また通過ペアーの設定は`BinanceConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BinanceConfig.BTCUSDT` 本番用BTCUSDTペアー（現物）
* `BinanceConfig.TEST_BTCUSDT`　テストネット用BTCUSDTペアー（現物）

## 本サンプルの機能

本サンプルでは以下を行います。

* オーダーの発行(Limit, Market)
* オーダーの検索
* オーダーのキャンセル
* 資産残高の確認

### 利用している関数
* オーダー発行
  * `limit_order(side, price, size)` リミットオーダーの発行
  * `market_order(side, size)`　マーケットオーダーの発行
* オーダー状況確認
  * `order_status`　オーダーステータスの確認（約定済みも含めた直近のもの）
  * `open_orders`　未約定一覧
* オーダーキャンセル
  * `cancel_order(order_id)`　指定したIDのオーダキャンセル
  * `cancel_all_orders()`　未約定のオーダーのキャンセル
* アカウント情報
  * `account`　アカウント情報（残高など）

## API KEYについて（注：まずはテストネットで試してください）
動作させるためにはAPI KEYが必要です。
ローカル環境では、以下の値を環境変数に指定してください。
* BINANCE_API_KEY   APIキー
* BINANCE_API_SERECT  シークレット

Google Clabでは、userdataに設定してください（Colabのページの左側ツールバーにある🔑マークをクリック）。

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。インストール後ランタイムの再起動が必要になる場合があります。

In [1]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。そのためバージョン指定しています。
! pip install --upgrade rbot==0.2.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 23.3 MB/s eta 0:00:00


In [ ]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html

## Python コード本体

### 必要ライブラリのインポート

In [1]:
import rbot
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
from rbot import BinanceMarket

# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig

from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

rbot version = 0.2.6


In [2]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### BinanceMarketオブジェクトの生成

引数にBinanceConfigを設定してオブジェクトを生成します。

In [ ]:
# BinanceMarketクラスのインスタンスを生成(テストネットのBTCUSDT現物取引の設定)
config = BinanceConfig.TEST_BTCUSDT

# Google Colabの場合は、userdataからAPIキーを取得
# 通常は、環境変数に設定してください。
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    # Google Colabの場合は、APIキーを入力してください。
    config.api_key = userdata.get("BINANCE_API_KEY")
    config.api_secret = userdata.get("BINANCE_API_SECRET")

# json2htmlを利用して、設定情報を表示
HTML(json2html.convert(config.__str__()))

In [4]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

# BinanceMarketのインスタンスを表示すると格納されているデータの情報が表示されます。
market

start,end
1700896516696000,1700898065939000
"""2023-11-25T07:15:16.696000""","""2023-11-25T07:41:05.939000"""
days=,0


In [5]:
from time import sleep

# WebSocketを利用して、取引所の情報を取得する
# CloabではWebSocketは動作しないので、その場合は、次のセルのbid_edge, ask_edgeを手動で入力する（現在の価格を両方にセットすればOK)
market.start_market_stream()

sleep(1) # データが揃うまで少し待つ

In [6]:
# 現在の板情報を取り出す
bids, asks, = market.board

bid_edge = bids['price'][0]
ask_edge = asks['price'][0]

print(bid_edge, ask_edge, bids['size'][0], asks['size'][0])

37763.16 37764.68 0.01166 0.00808


In [7]:
# 一旦　過去のオーダーを全部キャンセルする
market.cancel_all_orders()

[{"symbol":"BTCUSDT","create_time":1700898088326000,"status":"Canceled","order_id":"7798687","client_order_id":"erU2S6RGcCTpoH2a2DXNZ3","order_side":"Buy","order_type":"Limit","order_price":"37271.11000000","order_size":"0.00100000","remain_size":"0.00100000","transaction_id":"","update_time":0,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"","is_maker":false,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"0.0","lock_home_change":"0.0","lock_foreign_change":"0.0"}]

## オーダー発行テスト

### Limit order

In [ ]:
# 現在のアカウント情報を取り出す
account = market.account

HTML(json2html.convert(account.__str__()))

In [9]:
# accountにはすべての通貨の残高が入っているので、BTCとUSDTの残高を表示する
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00900000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9697.98517000","locked":"0.00000000"}


In [10]:
# 板より５００ドル安いところへ0.0001BTCの買い注文を出す
buy_limit_order = market.limit_order("BUY", bid_edge - 500, 0.001)

HTML(json2html.convert(buy_limit_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700898095803000,New,7799021,IXSbBJozCp9E4xbAOj1YgG,Buy,Limit,37263.16000000,0.00100000,0.00100000,,0,0.0,0.0,0.0,0.0,,False,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# オーダーTESTNET上の問題で１行しか表示されない場合は、すべてのオーダー履歴を表示して該当のオーダー状況を確認する。
HTML(json2html.convert(market.order_status.__str__()))

In [12]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず。(BINANCE TESTNETでは最初の１つしか表示されない問題あり)
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# 板より５００ドル高いところへ0.0001BTCの売り注文を出す
sell_limit_order = market.limit_order("SELL", ask_edge + 500, 0.001)

HTML(json2html.convert(sell_limit_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700898102337000,New,7799061,XmgunTxBfvRJ08NopktlfG,Sell,Limit,38264.68000000,0.00100000,0.00100000,,0,0.0,0.0,0.0,0.0,,False,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず（合計２つ）(BINANCE TESTNETでは最初の１つしか表示されない問題あり)
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# 最新の残高を確認。注文中の資産がLockされているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00800000","locked":"0.00100000"}
USDT残高 = {"asset":"USDT","free":"9660.72201000","locked":"37.26316000"}


In [16]:
# 売りオーダーをキャンセルする

id_to_cancel = buy_limit_order[0].order_id

print("キャンセルするオーダーID =", id_to_cancel)

market.cancel_order(sell_limit_order[0].order_id)

キャンセルするオーダーID = 7799021


{"symbol":"BTCUSDT","create_time":1700898108665000,"status":"Canceled","order_id":"7799061","client_order_id":"MmBo9bz2troyNeYIprC1bL","order_side":"Sell","order_type":"Limit","order_price":"38264.68000000","order_size":"0.00100000","remain_size":"0.00100000","transaction_id":"","update_time":0,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"","is_maker":false,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"0.0","lock_home_change":"0.0","lock_foreign_change":"0.0"}

In [17]:
# キャンセルしたオーダーがなくなっていることを確認する。(BINANCE TESTNETでは最初の１つしか表示されない問題あり)
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# 最新の残高を確認。キャンセルされた分の資産のLockが解放される。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00900000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9660.72201000","locked":"37.26316000"}


## Market order

In [19]:
# market orderを出す

market_order = market.market_order("BUY", 0.001)   # 0.001BTCを市場価格で買う
HTML(json2html.convert(market_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700898119353000,Filled,7799133,1pEZ0HrCCjQaZYULKzkNg1,Buy,Market,0.00000000,0.00100000,0.00000000,1569335,0,37765.36000000,0.00100000,37.7653600000000000,0.00000000,BTC,False,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# 直近のオーダーを表示する。status "Filled"のオーダーが１つ追加されているはず。
# なお大きなオーダーを出すと複数に分割される場合がある。

HTML(json2html.convert(market.order_status.__str__()))

symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
BTCUSDT,7513729,-1,JbeRhtrx4eYTsjPLQK4xAK,37773.99000000,0.00100000,0.00100000,37.77399000,Filled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700827718543,1700827806990,True,1700827718543,0.00000000,NONE
BTCUSDT,7513970,-1,xRSJykRtc1na5F7b9cvIsZ,37974.00000000,0.00100000,0.00100000,37.97400000,Filled,GTC,LIMIT,SELL,0.00000000,0.00000000,1700827832755,1700829920736,True,1700827832755,0.00000000,NONE
BTCUSDT,7518625,-1,sR3KMF5k1p1ZkreO6IbzA8,37848.34000000,0.00100000,0.00100000,37.84834000,Filled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700829281001,1700829808228,True,1700829281001,0.00000000,NONE
BTCUSDT,7518630,-1,B5TTCU7yVCV8F53WXy4dp1,38048.35000000,0.00100000,0.00100000,38.04835000,Filled,GTC,LIMIT,SELL,0.00000000,0.00000000,1700829282424,1700829920736,True,1700829282424,0.00000000,NONE
BTCUSDT,7519645,-1,MIDNtgQy58kqsBRkLMakDj,37841.24000000,0.00100000,0.00100000,37.84124000,Filled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700829590899,1700829808228,True,1700829590899,0.00000000,NONE
BTCUSDT,7519652,-1,X4dzKf5YspjYKSjM2C2Yfh,38041.31000000,0.00100000,0.00100000,38.04131000,Filled,GTC,LIMIT,SELL,0.00000000,0.00000000,1700829591968,1700829920736,True,1700829591968,0.00000000,NONE
BTCUSDT,7530348,-1,VdCXGtM0YeidZRC55BPkKD,37711.11000000,0.00100000,0.00100000,37.71111000,Filled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700832722710,1700833328005,True,1700832722710,0.00000000,NONE
BTCUSDT,7530354,-1,z4mYa9GbTXwDJLHJel8CxI,37911.12000000,0.00100000,0.00000000,0.00000000,Canceled,GTC,LIMIT,SELL,0.00000000,0.00000000,1700832724064,1700834969597,True,1700832724064,0.00000000,NONE
BTCUSDT,7530945,-1,8Z31w8XVZyA5o0RVLtG7GQ,37691.09000000,0.00100000,0.00100000,37.69109000,Filled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700832883494,1700833340694,True,1700832883494,0.00000000,NONE
BTCUSDT,7534043,-1,dpB4keSJA3HsMCnhs8zIsC,37607.59000000,0.00100000,0.00000000,0.00000000,Canceled,GTC,LIMIT,BUY,0.00000000,0.00000000,1700833719784,1700834969597,True,1700833719784,0.00000000,NONE


In [21]:
# 最新の残高を確認。約定した分だけBTCが増えて、USDTの残高が減っているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.01000000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9622.95665000","locked":"37.26316000"}
